# Predicting NBA players positions using Keras #

In this notebook we will build a neural net to predict the positions of NBA players using the [Keras](https://keras.io) library.

In [654]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [655]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, StandardScaler

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

## Data preparation ##

We will use the Kaggle dataset ["NBA Players stats since 1950"](https://www.kaggle.com/drgilermo/nba-players-stats), with stats for all players since 1950. We will take special interest in how the pass of time affects to the position of each player, and the definition of the positions themselves (a Small Forward, for example, was absolutely different in the 60's to what it is now)

In [656]:
stats = pd.read_csv(r'data/Seasons_Stats.csv', index_col=0)

The file ```Seasons_Stats.csv``` contains the statics of all players since 1950. First, we drop a couple of blank columns, and the "Tm" column, that contains the team.

In [830]:
stats = pd.read_csv(r'data/Seasons_Stats.csv', index_col=0)
stats_clean = stats.drop(['blanl', 'blank2', 'Tm'], axis=1)

In [831]:
stats_clean.head()

,Year,Player,Pos,Age,G,GS,MP,PER,TS%,3PAr,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1950.0,Curly Armstrong,G-F,31.0,63.0,NaN,NaN,NaN,0.368,NaN,...,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1950.0,Cliff Barker,SG,29.0,49.0,NaN,NaN,NaN,0.435,NaN,...,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,1950.0,Leo Barnhorst,SF,25.0,67.0,NaN,NaN,NaN,0.394,NaN,...,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,1950.0,Ed Bartels,F,24.0,15.0,NaN,NaN,NaN,0.312,NaN,...,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0
4,1950.0,Ed Bartels,F,24.0,13.0,NaN,NaN,NaN,0.308,NaN,...,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0


A second file, ```players.csv```, contains static information for each player, as height, weight, etc.

In [835]:
players = pd.read_csv(r'data/players.csv', index_col=0)
players.head(10)

,Player,height,weight,collage,born,birth_city,birth_state
0,Curly Armstrong,180.0,77.0,Indiana University,1918.0,NaN,NaN
1,Cliff Barker,188.0,83.0,University of Kentucky,1921.0,Yorktown,Indiana
2,Leo Barnhorst,193.0,86.0,University of Notre Dame,1924.0,NaN,NaN
3,Ed Bartels,196.0,88.0,North Carolina State University,1925.0,NaN,NaN
4,Ralph Beard,178.0,79.0,University of Kentucky,1927.0,Hardinsburg,Kentucky
5,Gene Berce,180.0,79.0,Marquette University,1926.0,NaN,NaN
6,Charlie Black,196.0,90.0,University of Kansas,1921.0,Arco,Idaho
7,Nelson Bobb,183.0,77.0,Temple University,1924.0,Philadelphia,Pennsylvania
8,Jake Bornheimer,196.0,90.0,Muhlenberg College,1927.0,New Brunswick,New Jersey
9,Vince Boryla,196.0,95.0,University of Denver,1927.0,East Chicago,Indiana


We merge both tables, and do some data cleaning:

* Keep only players with more than 400 minutes for each season (with a 82 games regular season, thats around 5 minutes per game. Players with less than that will be only anecdotical, and will distort the analysis).
* Replace the \* sign in some of the names
* For the stats that represent total values (others, as TS%, represent percentages), we will take the values per 36 minutes. The reason is to judge every player according to his characteristics, not the time he was on the floor.

In [839]:
data = pd.merge(stats_clean, players[['Player', 'height', 'weight']], left_on='Player', right_on='Player', right_index=False,
      how='left', sort=False).fillna(value=0)
data = data[~(data['Pos']==0) & (data['MP'] > 200)]
data.reset_index(inplace=True, drop=True)
data['Player'] = data['Player'].str.replace('*','')

totals = ['PER', 'OWS', 'DWS', 'WS', 'OBPM', 'DBPM', 'BPM', 'VORP', 'FG', 'FGA', '3P', '3PA', '2P', '2PA', 'FT', 'FTA',
         'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']

for col in totals:
    data[col] = 36 * data[col] / data['MP']

In [840]:
data.tail()

,Year,Player,Pos,Age,G,GS,MP,PER,TS%,3PAr,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,height,weight
19695,2017.0,Thaddeus Young,PF,28.0,74.0,74.0,2237.0,0.239785,0.562,0.172,...,5.117568,7.225749,1.963344,1.834600,0.482789,1.544926,2.172553,13.099687,203.0,100.0
19696,2017.0,Cody Zeller,PF,24.0,62.0,58.0,1725.0,0.348522,0.604,0.002,...,5.634783,8.452174,2.066087,1.293913,1.210435,1.356522,3.944348,13.335652,213.0,108.0
19697,2017.0,Tyler Zeller,C,27.0,51.0,5.0,525.0,0.891429,0.508,0.006,...,5.554286,8.502857,2.880000,0.480000,1.440000,1.371429,4.182857,12.205714,213.0,114.0
19698,2017.0,Paul Zipser,SF,22.0,44.0,18.0,843.0,0.294662,0.503,0.448,...,4.697509,5.338078,1.537367,0.640569,0.683274,1.708185,3.330961,10.249110,203.0,97.0
19699,2017.0,Ivica Zubac,C,19.0,38.0,11.0,609.0,1.004926,0.547,0.013,...,6.975369,9.399015,1.773399,0.827586,1.950739,1.773399,3.901478,16.788177,216.0,120.0


We will train a neural network with this data, to try to predict the position of each player.

A way we didn't follow was to transform the positions into numbers from 1 to 5 (1 for a PG, 2 for a SG, 1.5 for a PG-SG, and so on, until 5 for a C), and use the network for regression instead of classification. But we wanted to see if the network was able to predict labels as "SG-PF", so we decided to work with the categorical labels. Another reason is that this makes this study more easily portable to other areas.

We convert our DataFrame into a matrix X with the inputs, and a vector y with the labels. We scale the inputs and encode the outputs into dummy variables using the corresponding ```sklearn``` utilities.

Instead of a stochastic partition, we decided to use the 2017 season as our test data, and all the previous as the train set.

In [842]:
X = data.drop(['Player', 'Pos', 'G', 'GS', 'MP'], axis=1).as_matrix()
y = data['Pos'].as_matrix()

encoder = LabelBinarizer()
y_cat = encoder.fit_transform(y)
nlabels = len(encoder.classes_)

scaler =StandardScaler()
Xnorm = scaler.fit_transform(X)

stats2017 = (data['Year'] == 2017)
X_train = Xnorm[~stats2017]
y_train = y_cat[~stats2017]
X_test = Xnorm[stats2017]
y_test = y_cat[stats2017]

## Neural network training ##

We build using Keras (with Tensorflow as beckend) a neural network with two hidden layers. We will use relu activations, except for the last one, where we use a softmax to properly obtain the label probability. We will use a 20% of the data as a validation set, to make sure we are not overfitting.

In [851]:
model = Sequential()
model.add(Dense(40, activation='relu', input_dim=46))
model.add(Dropout(0.5))
model.add(Dense(30, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(nlabels, activation='softmax'))

In [853]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [854]:
# x_train and y_train are Numpy arrays --just like in the Scikit-Learn API.
model.fit(X_train, y_train, epochs=200, batch_size=128, validation_split=0.2, verbose=1)

Train on 15378 samples, validate on 3845 samples
Epoch 1/200
15378/15378 [==============================] - 1s - loss: 2.8803 - acc: 0.1578 - val_loss: 1.9745 - val_acc: 0.4107
Epoch 2/200
15378/15378 [==============================] - 0s - loss: 2.0452 - acc: 0.3210 - val_loss: 1.4504 - val_acc: 0.4661
Epoch 3/200
15378/15378 [==============================] - 0s - loss: 1.6946 - acc: 0.3707 - val_loss: 1.2529 - val_acc: 0.5394
Epoch 4/200
15378/15378 [==============================] - 0s - loss: 1.5110 - acc: 0.4004 - val_loss: 1.1616 - val_acc: 0.5664
Epoch 5/200
15378/15378 [==============================] - 0s - loss: 1.3971 - acc: 0.4282 - val_loss: 1.1100 - val_acc: 0.6031
Epoch 6/200
15378/15378 [==============================] - 0s - loss: 1.3392 - acc: 0.4400 - val_loss: 1.0867 - val_acc: 0.6343
Epoch 7/200
15378/15378 [==============================] - 0s - loss: 1.2855 - acc: 0.4607 - val_loss: 1.0710 - val_acc: 0.6502
Epoch 8/200
15378/15378 [==============================

15378/15378 [==============================] - 0s - loss: 0.8659 - acc: 0.6550 - val_loss: 0.8731 - val_acc: 0.6793
Epoch 128/200
15378/15378 [==============================] - 0s - loss: 0.8650 - acc: 0.6513 - val_loss: 0.8644 - val_acc: 0.7009
Epoch 129/200
15378/15378 [==============================] - 0s - loss: 0.8682 - acc: 0.6530 - val_loss: 0.8671 - val_acc: 0.6921
Epoch 130/200
15378/15378 [==============================] - 0s - loss: 0.8673 - acc: 0.6528 - val_loss: 0.8667 - val_acc: 0.7009
Epoch 131/200
15378/15378 [==============================] - 0s - loss: 0.8637 - acc: 0.6570 - val_loss: 0.8679 - val_acc: 0.7004
Epoch 132/200
15378/15378 [==============================] - 0s - loss: 0.8625 - acc: 0.6587 - val_loss: 0.8677 - val_acc: 0.6941
Epoch 133/200
15378/15378 [==============================] - 0s - loss: 0.8666 - acc: 0.6592 - val_loss: 0.8644 - val_acc: 0.7009
Epoch 134/200
15378/15378 [==============================] - 0s - loss: 0.8608 - acc: 0.6572 - val_loss:

In [855]:
model.test_on_batch(X_test, y_test, sample_weight=None)

[0.74140817, 0.69601679]

The model performs well both for the validation and the test sets (65% might not seem a lot, but it is satisfying enough for our problem, where all the labels are very subjective (Was Larry Bird a "SM-PF" or a "PF-SF"? Nobody can tell).

Now we train again the model, using all the training data (we will still reserve the 2017 season out of the training).

In [856]:
# Production model, using all data
model.fit(X_train, y_train, epochs=200, batch_size=128, validation_split=0, verbose=1)

Epoch 1/200
19223/19223 [==============================] - 0s - loss: 0.8416 - acc: 0.6743     
Epoch 2/200
19223/19223 [==============================] - 0s - loss: 0.8484 - acc: 0.6783     
Epoch 3/200
19223/19223 [==============================] - 0s - loss: 0.8460 - acc: 0.6733     
Epoch 4/200
19223/19223 [==============================] - 0s - loss: 0.8332 - acc: 0.6786     
Epoch 5/200
19223/19223 [==============================] - 0s - loss: 0.8373 - acc: 0.6799     
Epoch 6/200
19223/19223 [==============================] - 0s - loss: 0.8416 - acc: 0.6742     
Epoch 7/200
19223/19223 [==============================] - 0s - loss: 0.8364 - acc: 0.6770     
Epoch 8/200
19223/19223 [==============================] - 0s - loss: 0.8407 - acc: 0.6744     
Epoch 9/200
19223/19223 [==============================] - 0s - loss: 0.8351 - acc: 0.6754     
Epoch 10/200
19223/19223 [==============================] - 0s - loss: 0.8304 - acc: 0.6786     
Epoch 11/200
19223/19223 [=============

19223/19223 [==============================] - 0s - loss: 0.7768 - acc: 0.7063     
Epoch 169/200
19223/19223 [==============================] - 0s - loss: 0.7809 - acc: 0.7037     
Epoch 170/200
19223/19223 [==============================] - 0s - loss: 0.7836 - acc: 0.7021     
Epoch 171/200
19223/19223 [==============================] - 0s - loss: 0.7808 - acc: 0.7041     
Epoch 172/200
19223/19223 [==============================] - 0s - loss: 0.7832 - acc: 0.7043     
Epoch 173/200
19223/19223 [==============================] - 0s - loss: 0.7802 - acc: 0.7022     
Epoch 174/200
19223/19223 [==============================] - 0s - loss: 0.7789 - acc: 0.7025     
Epoch 175/200
19223/19223 [==============================] - 0s - loss: 0.7811 - acc: 0.7045     
Epoch 176/200
19223/19223 [==============================] - 0s - loss: 0.7817 - acc: 0.7039     
Epoch 177/200
19223/19223 [==============================] - 0s - loss: 0.7835 - acc: 0.7069     
Epoch 178/200
19223/19223 [=======

## Predicting the positions of the First NBA Team of 2017 ##

As a first test of the model, we will predict the positions of the player in the First NBA Team of 2017

In [857]:
first_team_members = ['Russell Westbrook', 'James Harden', 'Anthony Davis', 'LeBron James', 'Kawhi Leonard']
first_team_stats = data[[((x[1]['Player'] in first_team_members) & (x[1]['Year']==2017)) for x in data.iterrows()]]
first_team_stats

,Year,Player,Pos,Age,G,GS,MP,PER,TS%,3PAr,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,height,weight
19326,2017.0,Anthony Davis,C,23.0,75.0,75.0,2708.0,0.365583,0.579,0.088,...,9.465288,11.778434,2.087149,1.249631,2.220089,2.406204,2.233383,27.903988,206.0,97.0
19399,2017.0,James Harden,PG,27.0,81.0,81.0,2947.0,0.333492,0.613,0.493,...,6.889718,8.050221,11.067526,1.465898,0.451985,5.668137,2.626400,28.780455,196.0,99.0
19444,2017.0,LeBron James,SF,32.0,74.0,74.0,2794.0,0.347888,0.619,0.254,...,6.996421,8.246242,8.323550,1.185397,0.566929,3.904080,1.726557,25.176807,203.0,113.0
19486,2017.0,Kawhi Leonard,SF,25.0,74.0,74.0,2474.0,0.400162,0.611,0.294,...,5.092967,6.257074,3.783347,1.920776,0.800323,2.240905,1.775263,27.472918,201.0,104.0
19671,2017.0,Russell Westbrook,PG,28.0,81.0,81.0,2802.0,0.393148,0.554,0.300,...,9.340471,11.100642,10.792291,1.708779,0.398287,5.627409,2.441113,32.865096,190.0,90.0


In [858]:
pd.DataFrame(index=first_team_stats.loc[:, 'Player'].values, data={'Real': first_team_stats.loc[:, 'Pos'].values,
    'Predicted':encoder.inverse_transform(model.predict(Xnorm[first_team_stats.index, :]))})

,Predicted,Real
Anthony Davis,PF,C
James Harden,PG,PG
LeBron James,PF,SF
Kawhi Leonard,SF,SF
Russell Westbrook,PG,PG


The model gets right four of the five. It's even more interesting that the one that gets wrong, Anthony Davis, can play in both PF and C positions, and that in the last season, he played more as a Power Forward than as a Center, as the model predicts:

[New Orleans Pelicans Depth Chart - 2016-17](http://www.espn.com/nba/team/depth/_/name/no/new-orleans-pelicans).

## Predicting the positions of the NBA MVP ##

We will use now the model to predict the positions of all the NBA MVP since the creation of the award, in 1956.

In [859]:
mvp = [(1956, 'Bob Pettit'), (1957, 'Bob Cousy'), (1958, 'Bill Russell'), (1959, 'Bob Pettit'), 
(1960, 'Wilt Chamberlain'), (1961, 'Bill Russell'), (1962, 'Bill Russell'), (1963, 'Bill Russell'),
(1964, 'Oscar Robertson'), (1965, 'Bill Russell'), (1966, 'Wilt Chamberlain'), (1967, 'Wilt Chamberlain'),
(1968, 'Wilt Chamberlain'), (1969, 'Wes Unseld'), (1970, 'Willis Reed'), (1971, 'Lew Alcindor'), 
(1972, 'Kareem Abdul-Jabbar'), (1973, 'Dave Cowens'), (19704, 'Kareem Abdul-Jabbar'), (1975, 'Bob McAdoo'),
(1976, 'Kareem Abdul-Jabbar'), (1977, 'Kareem Abdul-Jabbar'), (1978, 'Bill Walton'), (1979, 'Moses Malone'), 
(1980, 'Kareem Abdul-Jabbar'), (1981, 'Julius Erving'), (1982, 'Moses Malone'), (1983, 'Moses Malone'), 
(1984, 'Larry Bird'), (1985, 'Larry Bird'), (1986, 'Larry Bird'), (1987, 'Magic Johnson'), 
(1988, 'Michael Jordan'), (1989, 'Magic Johnson'), (1990, 'Magic Johnson'), (1991, 'Michael Jordan'),
(1992, 'Michael Jordan'), (1993, 'Charles Barkley'), (1994, 'Hakeem Olajuwon'), (1995, 'David Robinson'),  
(1996, 'Michael Jordan'), (1997, 'Karl Malone'), (1998, 'Michael Jordan'), (1999, 'Karl Malone'), 
(2000, 'Shaquille O\'Neal'), (2001, 'Allen Iverson'), (2002, 'Tim Duncan'), (2003, 'Tim Duncan'), 
(2004, 'Kevin Garnett'), (2005, 'Steve Nash'), (2006, 'Steve Nash'), (2007, 'Dirk Nowitzki'), 
(2008, 'Kobe Bryant'), (2009, 'LeBron James'), (2010, 'LeBron James'), (2011, 'Derrick Rose'), 
(2012, 'LeBron James'), (2013, 'LeBron James'), (2014, 'Kevin Durant'), (2015, 'Stephen Curry'),
(2016, 'Stephen Curry')]

In [860]:
mvp_stats = pd.concat([data[(data['Player'] == x[1]) & (data['Year']==x[0])] for x in mvp], axis=0)

In [861]:
mvp_stats

,Year,Player,Pos,Age,G,GS,MP,PER,TS%,3PAr,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,height,weight
497,1956.0,Bob Pettit,C,23.0,72.0,0.0,2794.0,0.351754,0.502,0.000,...,0.000000,14.997853,2.435218,0.000000,0.000000,0.000000,2.602720,23.823908,206.0,92.0
536,1957.0,Bob Cousy,PG,28.0,64.0,0.0,2364.0,0.319797,0.452,0.000,...,0.000000,4.705584,7.279188,0.000000,0.000000,0.000000,2.040609,20.086294,185.0,79.0
688,1958.0,Bill Russell,C,23.0,69.0,0.0,2640.0,0.310909,0.465,0.000,...,0.000000,21.327273,2.754545,0.000000,0.000000,0.000000,2.468182,15.572727,208.0,97.0
784,1959.0,Bob Pettit,PF,26.0,72.0,0.0,2873.0,0.353359,0.519,0.000,...,0.000000,14.810999,2.769231,0.000000,0.000000,0.000000,2.506091,26.376610,206.0,92.0
827,1960.0,Wilt Chamberlain,C,23.0,72.0,0.0,3338.0,0.301977,0.493,0.000,...,0.000000,20.933493,1.811863,0.000000,0.000000,0.000000,1.617735,29.194727,216.0,124.0
992,1961.0,Bill Russell,C,26.0,78.0,0.0,3458.0,0.188433,0.454,0.000,...,0.000000,19.447079,2.790052,0.000000,0.000000,0.000000,1.613650,13.762869,208.0,97.0
1095,1962.0,Bill Russell,C,27.0,76.0,0.0,3433.0,0.203437,0.489,0.000,...,0.000000,18.770754,3.575881,0.000000,0.000000,0.000000,2.170696,15.058549,208.0,97.0
1208,1963.0,Bill Russell,C,28.0,78.0,0.0,3500.0,0.187200,0.464,0.000,...,0.000000,18.956571,3.579429,0.000000,0.000000,0.000000,1.944000,13.464000,208.0,97.0
1320,1964.0,Oscar Robertson,PG,25.0,79.0,0.0,3559.0,0.279180,0.576,0.000,...,0.000000,7.920202,8.779994,0.000000,0.000000,0.000000,2.832256,25.085698,196.0,92.0
1440,1965.0,Bill Russell,C,30.0,78.0,0.0,3466.0,0.202539,0.472,0.000,...,0.000000,19.506059,4.258511,0.000000,0.000000,0.000000,2.118869,11.446047,208.0,97.0


In [862]:
mvp_pred = pd.DataFrame(index=mvp_stats.loc[:, 'Player'].values, data={'Real': mvp_stats.loc[:, 'Pos'].values,
    'Predicted':encoder.inverse_transform(model.predict(Xnorm[mvp_stats.index, :]))})

In [737]:
mvp_pred

,Predicted,Real
Bob Pettit,C,C
Bob Cousy,PG,PG
Bill Russell,C,C
Bob Pettit,C,PF
Wilt Chamberlain,C,C
Bill Russell,C,C
Bill Russell,C,C
Bill Russell,C,C
Oscar Robertson,SG,PG
Bill Russell,C,C


The model gets right most of the players, and the errors are always for a contiguous position (it is interesting that the model gets this right without having been provided with any information about the distances between the labels.)

# Does year metter? #

The definitions of a forward or a center are always changing: in the very recent years, there is, for example, a trend towards having scoring point guards (as Stephen Curry) and forwards that direct the game instead of the guard (as Lebron James). 
Also, the physical requirements are increasing, and a height that in the 50's could characterize you as a center will make you a forward today.

We will follow the first and last MVP's, Stephen Curry and Bob Pettit, and see where our model puts them in different years in the NBA history.

In [863]:
curry2017 = data[(data['Player'] == 'Stephen Curry') & (data['Year']==2017)] 
pettit1956 = data[(data['Player'] == 'Bob Pettit') & (data['Year']==1956)]

In [864]:
time_travel_curry = pd.concat([curry2017 for year in range(1956, 2018)], axis=0)
time_travel_curry['Year'] = range(1956, 2018)

X = time_travel_curry.drop(['Player', 'Pos', 'G', 'GS', 'MP'], axis=1).as_matrix()
y = time_travel_curry['Pos'].as_matrix()

y_cat = encoder.transform(y)
Xnorm = scaler.transform(X)

time_travel_curry_pred = pd.DataFrame(index=time_travel_curry.loc[:, 'Year'].values, 
                                data={'Real': time_travel_curry.loc[:, 'Pos'].values,
    'Predicted':encoder.inverse_transform(model.predict(Xnorm))})


time_travel_pettit = pd.concat([pettit1956 for year in range(1956, 2018)], axis=0)
time_travel_pettit['Year'] = range(1956, 2018)

X = time_travel_pettit.drop(['Player', 'Pos', 'G', 'GS', 'MP'], axis=1).as_matrix()
y = time_travel_pettit['Pos'].as_matrix()

y_cat = encoder.transform(y)
Xnorm = scaler.transform(X)

time_travel_pettit_pred = pd.DataFrame(index=time_travel_pettit.loc[:, 'Year'].values, 
                                data={'Real': time_travel_pettit.loc[:, 'Pos'].values,
    'Predicted':encoder.inverse_transform(model.predict(Xnorm))})

In [865]:
pd.concat([time_travel_curry_pred,time_travel_pettit_pred],axis=1,keys=['Stephen Curry','Bob Pettit'])

Stephen Curry      Bob Pettit     
         Predicted Real  Predicted Real
1956            SG   PG          C    C
1957            SG   PG          C    C
1958            SG   PG          C    C
1959            SG   PG          C    C
1960            SG   PG          C    C
1961            SG   PG          C    C
1962            SG   PG          C    C
1963            SG   PG          C    C
1964            SG   PG          C    C
1965            SG   PG          C    C
1966            SG   PG          C    C
1967            SG   PG          C    C
1968            SG   PG         PF    C
1969            SG   PG         PF    C
1970            SG   PG         PF    C
1971            SG   PG         PF    C
1972            SG   PG         PF    C
1973            PG   PG         PF    C
1974            PG   PG         PF    C
1975            PG   PG         PF    C
1976            PG   PG         PF    C
1977            PG   PG         PF    C
1978            PG   PG         PF    C
1979            PG   PG         PF    C
1980            PG   PG         PF    C
1981            PG   PG         PF    C
1982            PG   PG         PF    C
1983            PG   PG         PF    C
1984            PG   PG         PF    C
1985            PG   PG         PF    C
...            ...  ...        ...  ...
1988            PG   PG         PF    C
1989            PG   PG         PF    C
1990            PG   PG         PF    C
1991            PG   PG         PF    C
1992            PG   PG         PF    C
1993            PG   PG         PF    C
1994            PG   PG         PF    C
1995            PG   PG         PF    C
1996            PG   PG         PF    C
1997            PG   PG         PF    C
1998            PG   PG         PF    C
1999            PG   PG         PF    C
2000            PG   PG         PF    C
2001            PG   PG         PF    C
2002            PG   PG         PF    C
2003            PG   PG         PF    C
2004            PG   PG         PF    C
2005            PG   PG         PF    C
2006            PG   PG         PF    C
2007            PG   PG         PF    C
2008            PG   PG         PF    C
2009            PG   PG         PF    C
2010            PG   PG         PF    C
2011            PG   PG         PF    C
2012            PG   PG         PF    C
2013            PG   PG         PF    C
2014            PG   PG         PF    C
2015            PG   PG         PF    C
2016            PG   PG         PF    C
2017            PG   PG         PF    C

[62 rows x 4 columns]

Curry is labeled as a point guard (his real position) from 1973 until today, and as a shooting guard before that. Perhaps because of his heigh (191cm), or perhaps because he is too much of a scorer. Bob Pettit is labeled as a center until 1967, and as a power forward after that (he played both roles, but nowadays he would have difficulties to play as a center, and would be for sure a forward, perhaps even a small forward). 

# Changing positions #

Many players go towards more interior roles with age, as they lose velocity. We will follow two cases, Magic Johnson, and Michael Jordan. Both of them retired, and returned years later with more interior roles.

In [866]:
magic = data[(data['Player'] == 'Magic Johnson')] 
jordan = data[(data['Player'] == 'Michael Jordan')]

In [867]:
# Magic
X = magic.drop(['Player', 'Pos', 'G', 'GS', 'MP'], axis=1).as_matrix()
y = magic['Pos'].as_matrix()

y_cat = encoder.transform(y)
Xnorm = scaler.transform(X)

magic_pred = pd.DataFrame(index=magic.loc[:, 'Age'].values, 
                                data={'Real': magic.loc[:, 'Pos'].values,
    'Predicted':encoder.inverse_transform(model.predict(Xnorm))})

# Jordan
X = jordan.drop(['Player', 'Pos', 'G', 'GS', 'MP'], axis=1).as_matrix()
y = jordan['Pos'].as_matrix()

y_cat = encoder.transform(y)
Xnorm = scaler.transform(X)

jordan_pred = pd.DataFrame(index=jordan.loc[:, 'Age'].values, 
                                data={'Real': jordan.loc[:, 'Pos'].values,
    'Predicted':encoder.inverse_transform(model.predict(Xnorm))})

In [869]:
pd.concat([magic_pred,jordan_pred],axis=1,keys=['Magic Johnson','Michael Jordan'])

Magic Johnson      Michael Jordan     
         Predicted Real      Predicted Real
20.0            SF   SG            NaN  NaN
21.0            SG   SG             SG   SG
22.0            SG   SG             SG   SG
23.0            PG   SG             SG   SG
24.0            PG   PG             SG   SG
25.0            PG   PG             SG   SG
26.0            PG   PG             SG   SG
27.0            PG   PG             SG   SG
28.0            PG   PG             SG   SG
29.0            PG   PG             SG   SG
30.0            PG   PG            NaN  NaN
31.0            PG   PG             SG   SG
32.0           NaN  NaN             SG   SG
33.0           NaN  NaN             SG   SG
34.0           NaN  NaN             SG   SG
36.0            SG   PF            NaN  NaN
38.0           NaN  NaN             SF   SF
39.0           NaN  NaN             SF   SF

The model is able to detect the conversion of Jordan into a forward at the end of his career, but not the return of Magic as a power forward. Also, in his rookie season, he is classified as a small forward instead of as a shooting guard (Magic was clearly and outlier in the data, a 205cm point guard who could easily play in the five positions. It is even surprising that is properly labelled as a point guard during most of his career)

# How important are height and weight? #

A concern we have before training the model was that it would use the height and weight as the main classifiers, and that it would label incorrectly players as Magic Johnson (a 205 cm point guard), or Charles Barkley (a 196cm power forward). Almost surprisingly, it works properly on this two players.

We will use again the 2017 First NBA Team and play with the heights and weights of the players. Keeping constant all other statistics, we will change the height and weight and observe how the predicted positions change.

In [1]:
first_team_stats

NameError: name 'first_team_stats' is not defined

In [871]:
multiplier = np.arange(0.8,1.2,0.02)
growing_predicted = []

for p in first_team_stats.iterrows():
    growing = pd.concat([p[1].to_frame().T for x in multiplier], axis=0)
    growing['height'] = growing['height'] * multiplier
    growing['weight'] = growing['weight'] * (multiplier ** 3)

    X = growing.drop(['Player', 'Pos', 'G', 'GS', 'MP'], axis=1).as_matrix()
    y = growing['Pos'].as_matrix()

    y_cat = encoder.transform(y)
    Xnorm = scaler.transform(X)

    growing_predicted.append(pd.DataFrame(index=multiplier, data={'height': growing.loc[:, 'height'].values,
            'Real': growing.loc[:, 'Pos'].values, 'Predicted':encoder.inverse_transform(model.predict(Xnorm))}))

/usr/local/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [874]:
pd.concat(growing_predicted,axis=1,keys=first_team_stats['Player'])

Player Anthony Davis              James Harden              LeBron James       \
           Predicted Real  height    Predicted Real  height    Predicted Real   
0.80              PF    C   164.8           PG   PG   156.8           PG   SF   
0.82              PF    C  168.92           PG   PG  160.72           PG   SF   
0.84              PF    C  173.04           PG   PG  164.64           PG   SF   
0.86              PF    C  177.16           PG   PG  168.56           PG   SF   
0.88              PF    C  181.28           PG   PG  172.48           PG   SF   
0.90              PF    C   185.4           PG   PG   176.4           PG   SF   
0.92              PF    C  189.52           PG   PG  180.32           PG   SF   
0.94              PF    C  193.64           PG   PG  184.24           PG   SF   
0.96              PF    C  197.76           PG   PG  188.16           SF   SF   
0.98              PF    C  201.88           PG   PG  192.08           SF   SF   
1.00              PF    C     206           PG   PG     196           PF   SF   
1.02              PF    C  210.12           PG   PG  199.92           PF   SF   
1.04               C    C  214.24           PG   PG  203.84            C   SF   
1.06               C    C  218.36           PG   PG  207.76            C   SF   
1.08               C    C  222.48           PF   PG  211.68            C   SF   
1.10               C    C   226.6            C   PG   215.6            C   SF   
1.12               C    C  230.72            C   PG  219.52            C   SF   
1.14               C    C  234.84            C   PG  223.44            C   SF   
1.16               C    C  238.96            C   PG  227.36            C   SF   
1.18               C    C  243.08            C   PG  231.28            C   SF   

Player         Kawhi Leonard              Russell Westbrook              
        height     Predicted Real  height         Predicted Real height  
0.80     162.4            PG   SF   160.8                PG   PG    152  
0.82    166.46            PG   SF  164.82                PG   PG  155.8  
0.84    170.52            PG   SF  168.84                PG   PG  159.6  
0.86    174.58            PG   SF  172.86                PG   PG  163.4  
0.88    178.64            PG   SF  176.88                PG   PG  167.2  
0.90     182.7            SG   SF   180.9                PG   PG    171  
0.92    186.76            SG   SF  184.92                PG   PG  174.8  
0.94    190.82            SG   SF  188.94                PG   PG  178.6  
0.96    194.88            SG   SF  192.96                PG   PG  182.4  
0.98    198.94            SG   SF  196.98                PG   PG  186.2  
1.00       203            SF   SF     201                PG   PG    190  
1.02    207.06            SF   SF  205.02                PG   PG  193.8  
1.04    211.12            SF   SF  209.04                SF   PG  197.6  
1.06    215.18            PF   SF  213.06                PF   PG  201.4  
1.08    219.24            PF   SF  217.08                PF   PG  205.2  
1.10     223.3             C   SF   221.1                PF   PG    209  
1.12    227.36             C   SF  225.12                PF   PG  212.8  
1.14    231.42             C   SF  229.14                 C   PG  216.6  
1.16    235.48             C   SF  233.16                 C   PG  220.4  
1.18    239.54             C   SF  237.18                 C   PG  224.2

As we can see height matters, but it's not enough. Any player can be classified as a center if he is tall enough (very tall: Kawhi Leonard would need to be 221cm tall to be considered a center), but being short it's not enough to be considered a guard: a 165cm Anthony Davis would be still considered a power forward.